Modelo para anlisis de sentmientos para luego precedir si tiene sintomas de depresion.

In [1]:
from string import punctuation
import pandas as pd
import nltk
import re

In [2]:
# read the data from tweets_public.csv and create a dataframe
df = pd.read_csv('data/tweets_public.csv', encoding='utf-8')
df.head()

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,neutral,False,0,0,Trabajar en #Ryanair como #TMA: https://t.co/r...,NaN,Fri Nov 03 12:05:12 +0000 2017,926419989107798016,NaN,Madrid
1,neutral,True,0,0,@Iberia @FIONAFERRER Cuando gusten en Cancún s...,NaN,Sun Nov 26 18:40:28 +0000 2017,934854385577943041,NaN,Mexico City
2,negative,False,0,0,Sabiais que @Iberia te trata muy bien en santi...,NaN,Mon Dec 25 15:40:45 +0000 2017,945318406441635840,NaN,Madrid
3,negative,False,0,0,NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...,NaN,Mon Nov 06 14:18:35 +0000 2017,927540721296568320,NaN,Pacific Time (US & Canada)
4,positive,True,0,0,@cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...,NaN,Mon Jan 01 23:00:57 +0000 2018,947965901332197376,NaN,Buenos Aires


In [3]:
df.shape

(7867, 10)

# Preprocesamiento
Limpieza de datos

In [4]:
# create a new dataframe with only the text and airline_sentiment columns and tweet id with the name df_sentiment
df_sentiment = df[['text', 'airline_sentiment', 'tweet_id']]

In [5]:
# transform the text letters to lowercase
df_sentiment['text'] = df_sentiment['text'].str.lower()
df_sentiment.head()

C:\Users\raula\AppData\Local\Temp\ipykernel_7712\3390815067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['text'] = df_sentiment['text'].str.lower()


,text,airline_sentiment,tweet_id
0,trabajar en #ryanair como #tma: https://t.co/r...,neutral,926419989107798016
1,@iberia @fionaferrer cuando gusten en cancún s...,neutral,934854385577943041
2,sabiais que @iberia te trata muy bien en santi...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair.\n...,negative,927540721296568320
4,@cris_tortu @dakar @iberia @mitsubishi_es @bfg...,positive,947965901332197376


In [6]:
# Referencia: https://stackoverflow.com/questions/6718633/python-regular-expression-again-match-url
# remove the urls from the text but keep all the text after the url
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: re.split('http[s]*\S+', str(x))[0])
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar en #ryanair como #tma:,neutral,926419989107798016
1,@iberia @fionaferrer cuando gusten en cancún s...,neutral,934854385577943041
2,sabiais que @iberia te trata muy bien en santi...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair.\n...,negative,927540721296568320
4,@cris_tortu @dakar @iberia @mitsubishi_es @bfg...,positive,947965901332197376


In [7]:
# remove the punctuation from the text
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: ''.join(c for c in x if c not in punctuation))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar en ryanair como tma,neutral,926419989107798016
1,iberia fionaferrer cuando gusten en cancún se ...,neutral,934854385577943041
2,sabiais que iberia te trata muy bien en santia...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair\nb...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [8]:
# change the \n to a space
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: x.replace('\n', ' '))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar en ryanair como tma,neutral,926419989107798016
1,iberia fionaferrer cuando gusten en cancún se ...,neutral,934854385577943041
2,sabiais que iberia te trata muy bien en santia...,negative,945318406441635840
3,nunca nunca nunca pidáis el café de ryanair bu...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [9]:
# remove the stopwrods from the text
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('spanish'))
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
df_sentiment.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\raula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,airline_sentiment,tweet_id
0,trabajar ryanair tma,neutral,926419989107798016
1,iberia fionaferrer gusten cancún viaja disfrut...,neutral,934854385577943041
2,sabiais iberia trata bien santiago chilete cam...,negative,945318406441635840
3,nunca nunca nunca pidáis café ryanair bueno ve...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [10]:
# remove the emojis from the text

# regular expression pattern to remove emojis from text
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # faces
        u"\U0001F300-\U0001F5FF"  # simbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)

# use lambda function to remove the emojis from the text
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: emoji_pattern.sub(r'', x))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar ryanair tma,neutral,926419989107798016
1,iberia fionaferrer gusten cancún viaja disfrut...,neutral,934854385577943041
2,sabiais iberia trata bien santiago chilete cam...,negative,945318406441635840
3,nunca nunca nunca pidáis café ryanair bueno ve...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [11]:
# remove the numbers from the text
df_sentiment.loc[:, 'text'] = df_sentiment['text'].apply(lambda x: re.sub(r'\d+', '', x))
df_sentiment.head()

,text,airline_sentiment,tweet_id
0,trabajar ryanair tma,neutral,926419989107798016
1,iberia fionaferrer gusten cancún viaja disfrut...,neutral,934854385577943041
2,sabiais iberia trata bien santiago chilete cam...,negative,945318406441635840
3,nunca nunca nunca pidáis café ryanair bueno ve...,negative,927540721296568320
4,cristortu dakar iberia mitsubishies bfgoodrich...,positive,947965901332197376


In [12]:
df_sentiment = df[['text', 'airline_sentiment']]
# df_sentiment['label'] = df_sentiment['airline_sentiment'].apply(lambda x: '1 start' if x == 'negative' else '5 stars')
df_sentiment['label'] = df_sentiment['airline_sentiment'].apply(lambda x: 0 if x == 'negative' else 1)
df_sentiment = df_sentiment[['text', 'label']]
df_sentiment

C:\Users\raula\AppData\Local\Temp\ipykernel_7712\4261467530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['label'] = df_sentiment['airline_sentiment'].apply(lambda x: 0 if x == 'negative' else 1)


,text,label
0,Trabajar en #Ryanair como #TMA: https://t.co/r...,1
1,@Iberia @FIONAFERRER Cuando gusten en Cancún s...,1
2,Sabiais que @Iberia te trata muy bien en santi...,0
3,NUNCA NUNCA NUNCA pidáis el café de Ryanair.\n...,0
4,@cris_tortu @dakar @Iberia @Mitsubishi_ES @BFG...,1
...,...,...
7862,@Iberia @iberiaexpress especialistas en dejart...,0
7863,"Con @Iberia, mi destino a un solo click. ¡Dese...",1
7864,@Iberia Muy bien. Muchas gracias,1
7865,Es que volar con Ryanair es peor que irte a ch...,0


In [13]:
# save the dataframe to a csv file
df_sentiment.to_csv('data/clean_tweets_sentiment.csv', index=False)

# Hugging Face

In [14]:
model_preload1 = "nlptown/bert-base-multilingual-uncased-sentiment"
# model_preload = "distilbert-base-uncased"
# model_preload = "xlm-roberta-base"
# model_preload = "distilbert-base-multilingual-cased"
# model_preload = "bert-base-multilingual-cased"
model_preload =  'dccuchile/bert-base-spanish-wwm-cased'
# model_preload = "RoBERTa-base"

In [15]:

data = ["Amo el mundo en el que vivo", "Odio a todas las personas que conozco"]

In [16]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9905071258544922},
 {'label': 'NEGATIVE', 'score': 0.9892126321792603}]

In [17]:
sentiment_pipeline('no estoy normal')

[{'label': 'NEGATIVE', 'score': 0.9994358420372009}]

In [18]:
specific_model = pipeline("sentiment-analysis", model=model_preload1)
specific_model(data)

[{'label': '5 stars', 'score': 0.7264281511306763},
 {'label': '5 stars', 'score': 0.6389875411987305}]

In [19]:
specific_model('estoy normal')

[{'label': '3 stars', 'score': 0.3511374890804291}]

In [20]:
specific_model('estoy triste')

[{'label': '1 star', 'score': 0.5364329814910889}]

## Fine tuning

In [21]:
# import torch
# torch.cuda.is_available()

In [22]:
# torch.cuda.empty_cache()

In [23]:
from datasets import load_dataset
local_train = load_dataset("csv", data_files="data/clean_tweets_sentiment.csv", split="train[:70%]")
local_test = load_dataset("csv", data_files="data/clean_tweets_sentiment.csv", split="train[70%:]")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/raula/.cache/huggingface/datasets/csv/default-166099f8577deca8/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


Found cached dataset csv (C:/Users/raula/.cache/huggingface/datasets/csv/default-166099f8577deca8/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


In [24]:
print(local_train.shape)
print(local_test.shape)
print(local_train[0])
print(local_test[0])

(5507, 2)
(2360, 2)
{'text': 'Trabajar en #Ryanair como #TMA: https://t.co/ruUArBe1tO #empleo', 'label': 1}
{'text': '@radioledonline @Iberia @lauraluzo Sr. Gallego, por qué no pagáis las indemnizaciones por cancelaciones? Aviación C… https://t.co/pC3cvISmjH', 'label': 0}


In [25]:
small_train_dataset = local_train.shuffle(seed=42).select(
    list(list(range(300)))
)
small_test_dataset = local_test.shuffle(seed=42).select(list(list(range(100))))
# # split df_sentiment in 70% train and 30% test
# from sklearn.model_selection import train_test_split

# train, test = train_test_split(df_sentiment, test_size=0.3, random_state=42)
# train.shape, test.shape

In [26]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_preload)

In [27]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

# user the function preprocess_function to tokenize the train and test data
# tokenized_train = train['text'].apply(lambda x: tokenizer(x, truncation=True))
# tokenized_test = test['text'].apply(lambda x: tokenizer(x, truncation=True))
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [28]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## train the model

In [29]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_preload, num_labels=2)


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

In [30]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [31]:
from huggingface_hub import notebook_login
notebook_login()

In [32]:
from transformers import TrainingArguments, Trainer
 
repo_name = "raulangelj/huggingface_sentiment_analysis"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=25,
   per_device_eval_batch_size=25,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

c:\Users\raula\Documents\RAUL_ANGEL\UVG_COMPU\NOVENO_SEMESTRE\DISENO-E-INOVACION\analisis_sentimientos_depresion\raulangelj/huggingface_sentiment_analysis is already a clone of https://huggingface.co/raulangelj/huggingface_sentiment_analysis. Make sure you pull the latest changes with `repo.git_pull()`.


In [33]:
trainer.train()

C:\Users\raula\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\raula/.netrc


  0%|          | 0/24 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'train_runtime': 628.1913, 'train_samples_per_second': 0.955, 'train_steps_per_second': 0.038, 'train_loss': 0.6502002477645874, 'epoch': 2.0}


TrainOutput(global_step=24, training_loss=0.6502002477645874, metrics={'train_runtime': 628.1913, 'train_samples_per_second': 0.955, 'train_steps_per_second': 0.038, 'train_loss': 0.6502002477645874, 'epoch': 2.0})

In [34]:
trainer.evaluate()

  0%|          | 0/4 [00:00<?, ?it/s]

C:\Users\raula\AppData\Local\Temp\ipykernel_7712\1757224399.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.6655668020248413,
 'eval_accuracy': 0.59,
 'eval_f1': 0.6666666666666667,
 'eval_runtime': 2.4202,
 'eval_samples_per_second': 41.319,
 'eval_steps_per_second': 1.653,
 'epoch': 2.0}

In [36]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/raulangelj/huggingface_sentiment_analysis
   bad9ad3..998b40d  main -> main



In [37]:
data

['Amo el mundo en el que vivo', 'Odio a todas las personas que conozco']

In [38]:
from transformers import pipeline
 
sentiment_model = pipeline(model=repo_name)
sentiment_model(data)

[{'label': 'LABEL_1', 'score': 0.567038893699646},
 {'label': 'LABEL_1', 'score': 0.508574903011322}]

In [39]:
sentiment_model('no me siento bien')

[{'label': 'LABEL_0', 'score': 0.5060250759124756}]

In [40]:
sentiment_model('hoy es el mejor dia de mi vida')

[{'label': 'LABEL_1', 'score': 0.5540620684623718}]

In [41]:
sentiment_model('ya no puedo con mi vida, no quiero que me hablen')

[{'label': 'LABEL_0', 'score': 0.5562945008277893}]

# BERT

In [ ]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from textwrap import wrap

In [ ]:
# Inicialización
RANDOM_SEED = 42
MAX_LEN = 200
BATCH_SIZE = 16
# DATASET_PATH = '/content/drive/My Drive/videos/2020-07-20/BERT_sentiment_IMDB_Dataset.csv'
NCLASSES = 3

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# # Cargar dataset
# from google.colab import drive
# drive.mount('/content/drive')

# df_bert = pd.read_csv(DATASET_PATH)
# df_bert = df[0:10000]

df_bert = df_sentiment[['text', 'airline_sentiment']]

In [ ]:
print(df.head())
print(df.shape)
print("\n".join(wrap(df['text'][200])))

In [ ]:
# Reajustar dataset
df_bert['label'] = df_bert['airline_sentiment'].apply(lambda x: 1 if x == 'positive' else 0 if x == 'neutral' else -1)
df_bert = df_bert.drop(['airline_sentiment'], axis=1)
df_bert.head()

In [ ]:
# TOKENIZACIÓN
PRE_TRAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
# Ejemplo tokenización
sample_txt = 'Me gusto mucho esa pelicula!'
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print('Frase: ', sample_txt)
print('Tokens: ', tokens)
print('Tokens numéricos: ', token_ids)

In [ ]:
# Codificación para introducir a BERT
encoding = tokenizer.encode_plus(
    sample_txt,
    max_length = 10,
    truncation = True,
    add_special_tokens = True,
    return_token_type_ids = False,
    pad_to_max_length = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

In [ ]:
encoding.keys()

In [ ]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])

In [ ]:
# CREACIÓN DATASET

class TEXTDataset(Dataset):

  def __init__(self,texts,labels,tokenizer,max_len):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
      return len(self.texts)
    
  def __getitem__(self, item):
    text = str(self.texts[item])
    label = self.labels[item]
    encoding = tokenizer.encode_plus(
        text,
        max_length = self.max_len,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
        )
    

    return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'label': torch.tensor(label, dtype=torch.long)
      } 

In [ ]:
# Data loader:

def data_loader(df, tokenizer, max_len, batch_size):
  dataset = TEXTDataset(
      texts = df.text.to_numpy(),
      labels = df.label.to_numpy(),
      tokenizer = tokenizer,
      max_len = MAX_LEN
  )

  return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 4)

In [ ]:
df_train_bert, df_test_bert = train_test_split(df_bert, test_size = 0.2, random_state=RANDOM_SEED)

train_data_loader = data_loader(df_train_bert, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test_bert, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
# Bert Model
class BERTSentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BERTSentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, cls_output = self.bert(
        input_ids = input_ids,
        attention_mask = attention_mask
    )
    drop_output = self.drop(cls_output)
    return self.linear(drop_output)


In [ ]:
model = BERTSentimentClassifier(NCLASSES)
model = model.to(device)

In [ ]:
# print(model)

In [ ]:
# Training
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
# Iteración entrenamiento
def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for batch in data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    outputs = model(input_ids = input_ids, attention_mask = attention_mask)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)
    correct_predictions += torch.sum(preds == labels)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double()/n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      outputs = model(input_ids = input_ids, attention_mask = attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double()/n_examples, np.mean(losses)

In [ ]:
# Entrenamiento!!!

for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1} de {EPOCHS}')
  print('------------------')
  train_acc, train_loss = train_model(
      model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train_bert)
  )
  test_acc, test_loss = eval_model(
      model, test_data_loader, loss_fn, device, len(df_test_bert)
  )
  print(f'Entrenamiento: Loss: {train_loss}, accuracy: {train_acc}')
  print(f'Validación: Loss: {test_loss}, accuracy: {test_acc}')
  print('')

In [ ]:
def classifySentiment(text):
  encoding_review = tokenizer.encode_plus(
      text,
      max_length = MAX_LEN,
      truncation = True,
      add_special_tokens = True,
      return_token_type_ids = False,
      pad_to_max_length = True,
      return_attention_mask = True,
      return_tensors = 'pt'
      )
  
  input_ids = encoding_review['input_ids'].to(device)
  attention_mask = encoding_review['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)
  print('predicción = ', prediction)
  print("\n".join(wrap(text)))

